# Predicting Sale Price from NYC Property Sale Data
## Berkeley SAAS CX Fall 2020 Kaggle Competition
## Ming Fong and Yifan Zhang


Helpful:

https://www.kaggle.com/sahilrider/learn-regression-nyc#4.-Modelling

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import linear_model
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [2]:
# Display all columns of Dataframes
pd.set_option('display.max_columns', None)

In [3]:
X_train = pd.read_csv("data/train_features.csv")
y_train = pd.read_csv("data/train_targets.csv")
X_test = pd.read_csv("data/test_features.csv")
# sample_submission = pd.read_csv("output/sample_submission.csv")

In [4]:
# X_train.set_index("id")
# y_train.set_index("id")
# X_test.set_index("id")
# X_train.head()

In [5]:
# merge sale price onto feature data
df = X_train.merge(right = y_train, on = "id", how = "outer")

# train_merged = X_train.merge(right = y_train, on = "id")

In [6]:
# drop training rows with 0 gross or land sqft
# makes score worse

# df = df.drop(df[df["GROSS SQUARE FEET"] == 0].index)
# df = df.drop(df[df["LAND SQUARE FEET"] == 0].index)

In [7]:
df = pd.concat([df, X_test], axis=0, ignore_index = True)

In [8]:
df = df.set_index("id")

In [9]:
# "EASE-MENT" is an empty column, delete it
del df["EASE-MENT"]

# Delete date column
del df["SALE DATE"]

del df["LOT"]
del df["ADDRESS"]
del df["APARTMENT NUMBER"]

In [10]:
# Correlation between features

# corr = train_merged.corr()
# print(corr['SALE PRICE'].sort_values(ascending=False))
# sns.heatmap(corr)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37153 entries, train0 to test9288
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         37153 non-null  int64  
 1   NEIGHBORHOOD                    37153 non-null  object 
 2   BUILDING CLASS CATEGORY         37153 non-null  object 
 3   TAX CLASS AT PRESENT            37153 non-null  object 
 4   BLOCK                           37153 non-null  int64  
 5   BUILDING CLASS AT PRESENT       37153 non-null  object 
 6   ZIP CODE                        37153 non-null  int64  
 7   RESIDENTIAL UNITS               37153 non-null  int64  
 8   COMMERCIAL UNITS                37153 non-null  int64  
 9   TOTAL UNITS                     37153 non-null  int64  
 10  LAND SQUARE FEET                37153 non-null  float64
 11  GROSS SQUARE FEET               37153 non-null  float64
 12  YEAR BUILT                   

In [12]:
categorical_features = ["BOROUGH", "BUILDING CLASS CATEGORY", "TAX CLASS AT TIME OF SALE", "ZIP CODE"]

del df["BUILDING CLASS AT PRESENT"]
del df["TAX CLASS AT PRESENT"]
del df["BUILDING CLASS AT TIME OF SALE"]

# Too many features for location
del df["BLOCK"]
del df["NEIGHBORHOOD"]
# del df["ZIP CODE"]

In [13]:
# One-hot encoding
df = pd.get_dummies(df, columns = categorical_features)
df.info(verbose = True, memory_usage = True, null_counts = True)

             37153 non-null  uint8  
 37  BUILDING CLASS CATEGORY_26 OTHER HOTELS                               37153 non-null  uint8  
 38  BUILDING CLASS CATEGORY_27 FACTORIES                                  37153 non-null  uint8  
 39  BUILDING CLASS CATEGORY_28 COMMERCIAL CONDOS                          37153 non-null  uint8  
 40  BUILDING CLASS CATEGORY_29 COMMERCIAL GARAGES                         37153 non-null  uint8  
 41  BUILDING CLASS CATEGORY_30 WAREHOUSES                                 37153 non-null  uint8  
 42  BUILDING CLASS CATEGORY_31 COMMERCIAL VACANT LAND                     37153 non-null  uint8  
 43  BUILDING CLASS CATEGORY_32 HOSPITAL AND HEALTH FACILITIES             37153 non-null  uint8  
 44  BUILDING CLASS CATEGORY_33 EDUCATIONAL FACILITIES                     37153 non-null  uint8  
 45  BUILDING CLASS CATEGORY_34 THEATRES                                   37153 non-null  uint8  
 46  BUILDING CLASS CATEGORY_35 INDOOR PUBLIC AND CULTURAL FACILITIES   

# Regression

In [14]:
# Get the number of rows from the test data (should be 9289)

test_row_count = X_test.shape[0]
test_row_count

9289

In [15]:
# Split train and test data
from sklearn.model_selection import train_test_split

y = df["SALE PRICE"][:(df.shape[0] - test_row_count)]
X = df[:(df.shape[0] - test_row_count)].drop("SALE PRICE", 1)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
from sklearn.metrics import mean_squared_error
# Root mean squared error
def rmse(y_test,y_pred):
      return np.sqrt(mean_squared_error(y_test,y_pred))

## Linear Regression

In [17]:
# linear regression
linreg = linear_model.LinearRegression()
linreg.fit(X_train, y_train)
y_pred_lin = linreg.predict(X_valid)
rmse(y_valid, y_pred_lin)

8018162.3439041255

In [18]:
linear_submission = pd.DataFrame(linreg.predict(df[(df.shape[0] - test_row_count):].drop("SALE PRICE", 1)))
linear_submission.index = X_test["id"]
linear_submission.columns = ["SALE PRICE"]
linear_submission.to_csv("output/linear_submission.csv")
linear_submission.head()

,SALE PRICE
id,
test0,1.393442e+07
test1,6.799121e+05
test2,4.041972e+05
test3,1.922549e+06
test4,2.661554e+06


## LASSO Regression

In [93]:
from scipy.optimize import minimize 

In [94]:
# Lasso regression
def lasso(alpha, iterations = 50):
    lasso_regr = linear_model.Lasso(alpha = alpha, max_iter = iterations)
    lasso_regr.fit(X_train, y_train)
    Y_pred_lasso = lasso_regr.predict(X_valid)
    return rmse(y_valid,Y_pred_lasso)

In [95]:
lasso_res = minimize(lasso, x0 = 10)
lasso_res

      fun: 7717326.318858968
 hess_inv: array([[5.24520602]])
      jac: array([0.])
  message: 'Optimization terminated successfully.'
     nfev: 135
      nit: 9
     njev: 45
   status: 0
  success: True
        x: array([-3661.67016035])

In [96]:
lasso_regr = linear_model.Lasso(alpha = lasso_res.x[0], max_iter = 500)
lasso_regr.fit(X_train, y_train)
Y_pred_lasso = lasso_regr.predict(X_valid)
rmse(y_valid,Y_pred_lasso)

7786545.059912653

In [97]:
lasso_submission = pd.DataFrame(lasso_regr.predict(df[(df.shape[0] - test_row_count):].drop("SALE PRICE", 1)))
lasso_submission.index = X_test["id"]
lasso_submission.columns = ["SALE PRICE"]
lasso_submission[lasso_submission["SALE PRICE"] < 1000] = y_train.median()       # hard code set < 1000 to median
lasso_submission.to_csv("output/lasso_submission.csv")
lasso_submission.head()

,SALE PRICE
id,
test0,9.935354e+06
test1,1.168445e+06
test2,1.575592e+06
test3,8.742068e+05
test4,2.802492e+06


## Ridge Regression

In [116]:
# Ridge regression
ridge = linear_model.Ridge(alpha = 0.029, normalize = True)
ridge.fit(X_train, y_train)
Y_pred_ridge = ridge.predict(X_valid)
rmse(y_valid, Y_pred_ridge)

7990095.857629159

In [117]:
ridge_submission = pd.DataFrame(ridge.predict(df[(df.shape[0] - test_row_count):].drop("SALE PRICE", 1)))
ridge_submission.index = X_test["id"]
ridge_submission.columns = ["SALE PRICE"]
ridge_submission.to_csv("output/ridge_submission.csv")
ridge_submission.head()

,SALE PRICE
id,
test0,1.396353e+07
test1,6.846967e+05
test2,4.095589e+05
test3,1.877738e+06
test4,2.753596e+06


## Random Forest

In [89]:
from sklearn.ensemble import RandomForestRegressor

In [90]:
# Random forest
rf_regr = RandomForestRegressor()
rf_regr.fit(X_train, y_train)
Y_pred_rf = rf_regr.predict(X_valid)
rmse(y_valid, Y_pred_rf)

8669931.623188293

In [91]:
rf_submission = pd.DataFrame(rf_regr.predict(df[(df.shape[0] - test_row_count):].drop("SALE PRICE", 1)))
rf_submission.index = X_test["id"]
rf_submission.columns = ["SALE PRICE"]
rf_submission.to_csv("output/rf_submission.csv")
rf_submission.head()

,SALE PRICE
id,
test0,8.562035e+06
test1,6.330833e+05
test2,5.337176e+05
test3,1.182609e+06
test4,9.077677e+05


## XGBoost

Helpful:

https://www.kaggle.com/darkknight98/how-a-data-scientist-buys-a-house-a-tutorial

In [19]:
import xgboost
from sklearn.model_selection import RandomizedSearchCV

In [125]:
# regressor = xgboost.XGBRegressor()
# n_estimators = [100, 500, 900, 1100, 1500]
# max_depth = [2, 3, 5, 10, 15]
# booster = ['gbtree','gblinear']
# learning_rate = [0.05, 0.1, 0.15, 0.20]
# min_child_weight = [1, 2, 3, 4]
# base_score = [0.25, 0.5, 0.75, 1]

# # Define the grid of hyperparameters to search
# hyperparameter_grid = {
#     'n_estimators': n_estimators,
#     'max_depth': max_depth,
#     'learning_rate': learning_rate,
#     'min_child_weight': min_child_weight,
#     'booster': booster,
#     'base_score': base_score
#     }
# random_cv = RandomizedSearchCV(estimator=regressor,
#             param_distributions=hyperparameter_grid,
#             cv=5, n_iter=50,
#             scoring = 'neg_mean_absolute_error',n_jobs = -1,
#             verbose = 5, 
#             return_train_score = True,
#             random_state=42)

In [126]:
# random_cv.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  3.3min


In [ ]:
# random_cv.best_estimator_

In [20]:
# TODO tweak these params to find the best one (see code above)

regressor = xgboost.XGBRegressor(base_score=0.25, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=2,
             min_child_weight=1, monotone_constraints='()',
             n_estimators=900, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [21]:
regressor.fit(X_train, y_train)
print('R-squared score (training): {:.3f}'.format(regressor.score(X_train, y_train)))
regressor.fit(X_valid, y_valid)
print('R-squared score (test): {:.3f}'.format(regressor.score(X_valid, y_valid)))

R-squared score (training): 0.968
R-squared score (test): 0.993


In [70]:
xgb_pred = regressor.predict(df[(df.shape[0] - test_row_count):].drop("SALE PRICE", 1))
xgb_pred = pd.DataFrame(xgb_pred)

xgb_pred.index = X_test["id"]
xgb_pred.columns = ["SALE PRICE"]
xgb_pred[xgb_pred["SALE PRICE"] < 0] = y_train.median()       # hard code set < 0 to median
xgb_pred.to_csv("output/xgb_submission.csv")
xgb_pred.head()

,SALE PRICE
id,
test0,9.011357e+06
test1,6.629601e+05
test2,4.261658e+05
test3,1.224445e+06
test4,1.833526e+06


# Submit to Kaggle
Run this command to send a submission to the Kaggle competition

In [58]:
# !kaggle competitions submit -c saas-2020-fall-cx-kaggle-compeition -f output/xgb_submission.csv -m "xgb 1 (negatives = 10)"

Successfully submitted to SAAS 2020 Fall CX Kaggle Compeition

  0%|          | 0.00/176k [00:00<?, ?B/s]
 45%|████▌     | 80.0k/176k [00:00<00:00, 766kB/s]
 59%|█████▉    | 104k/176k [00:00<00:00, 238kB/s] 
 73%|███████▎  | 128k/176k [00:00<00:00, 164kB/s]
 82%|████████▏ | 144k/176k [00:00<00:00, 131kB/s]
 91%|█████████ | 160k/176k [00:00<00:00, 119kB/s]
100%|█████████▉| 176k/176k [00:01<00:00, 109kB/s]
100%|██████████| 176k/176k [00:03<00:00, 47.8kB/s]
